# Reconocimiento de rostros con OpenCV, Python, and deep learning

El reconocimiento facial con OpenCV, Python y deep learning es una técnica avanzada de visión artificial que permite a una computadora identificar y verificar rostros humanos en imágenes y videos. OpenCV es una biblioteca de procesamiento de imágenes y visión artificial que facilita la detección de rostros mediante técnicas de análisis de imágenes. Python es el lenguaje de programación utilizado para implementar y coordinar estos algoritmos, dado su versatilidad y la extensa disponibilidad de bibliotecas de deep learning. Finalmente, deep learning permite entrenar modelos basados en redes neuronales profundas que aprenden características faciales distintivas mediante "aprendizaje de métrica profunda" (deep metric learning), lo que permite clasificar y reconocer rostros de manera precisa y en tiempo real.

OpenCV, que es como los ojos de la computadora. Python es el lenguaje que le habla a la computadora para decirle qué hacer. Y deep learning es como el cerebro de la computadora, que aprende y se entrena para reconocer caras, como cuando aprendes a recordar la cara de un amigo.

Github: https://pyimagesearch.com/2018/06/18/face-recognition-with-opencv-python-and-deep-learning/

### Libreria dlib

Es una libreria popular para machine learning y visión artificial, conocida por sus implementaciones de algoritmos de detección de rostros, alineación facial y otras técnicas de procesamiento de imágenes.

In [1]:
conda install -c conda-forge dlib

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Admin\anaconda3

  added / updated specs:
    - dlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.13.0               |   py39hcbf5309_1         1.0 MB  conda-forge
    dlib-19.24.0               |   py39hf8509d4_0         4.4 MB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following NEW packages will be INSTALLED:

  dlib               conda-forge/win-64::dlib-19.24.0-py39hf8509d4_0
  python_abi         conda-forge/win-64::python_abi-3.9-2_cp39

The following packages will be UPDATED:

  conda              pkgs/main::conda-4.13.0-py39haa95532_0 --> conda-forge::conda-4.13.0-py39hcbf5309_1




conda-4.13.0

### Libreria face_recognition

face_recognition es una herramienta de Python basada en el reconocimiento facial que utiliza redes neuronales profundas para identificar y verificar rostros en imágenes y videos.Esta libreria permite realizar operaciones avanzadas de procesamiento de imágenes, como detección de rostros, reconocimiento de identidad y comparación de rostros. Su facilidad de uso y precisión hacen que face_recognition sea ampliamente utilizada en aplicaciones de seguridad, autenticación biométrica y análisis de datos visuales en tiempo real, simplificando la implementación de algoritmos complejos de visión artificial en proyectos de reconocimiento facial.

In [3]:

conda install -c conda-forge face_recognition

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Admin\anaconda3

  added / updated specs:
    - face_recognition


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    face_recognition-1.3.0     |     pyhd3deb0d_2          17 KB  conda-forge
    face_recognition_models-0.3.0|     pyh9f0ad1d_0        87.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        87.7 MB

The following NEW packages will be INSTALLED:

  face_recognition   conda-forge/noarch::face_recognition-1.3.0-pyhd3deb0d_2
  face_recognition_~ conda-forge/noarch::face_recognition_models-0.3.0-pyh9f0ad1d_0




face_recognition-1.3 | 17 KB     |            |   0% 
face_recognition-1.3 | 17 KB     | #########5 |  95% 
face_recognition-1.

In [5]:
#Para el procesamiento de imágenes y visión artificial en Python
!pip install opencv-python

In [2]:
# Libreria complementaria para trabajar con OpenCV, se usa para la rotación, el cambio de tamaño, la traslación y la detección de bordes en imágenes.
!pip install imutils

  Using cached imutils-0.5.4.tar.gz (17 kB)
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25872 sha256=17a7cb23174346c15ff17d2f92a107947d6da7b74d2772060ffa9e2441f50d30
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


# Dataset

##### Creación de imagenes sintéticas
Para esta parte se utilizará imágenes sintéticas generadas a partir de una sola foto por alumno del 10mo ciclo. Esto nos ayudará a crear un conjunto de imágenes más amplio para entrenar nuestro modelo de reconocimiento facial. Además usaremos fotos de actores reconocidos de peliculas en estreno.

In [ ]:
# Librería Keras (con ImageDataGenerator)
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os

In [ ]:
# Ruta a la carpeta del dataset
dataset_path = "dataset"

In [ ]:
# Configuración de aumentación de datos
datagen = ImageDataGenerator(
    rotation_range=15,         # Rotación hasta 15 grados
    width_shift_range=0.1,     # Desplazamiento horizontal
    height_shift_range=0.1,    # Desplazamiento vertical
    brightness_range=[0.8, 1.2],  # Variación de brillo
    zoom_range=0.2             # Zoom
)

In [ ]:
# Generar imágenes aumentadas para cada persona
for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
    if os.path.isdir(person_folder):
        # Cargar la imagen original de la persona
        original_image_path = os.path.join(person_folder, f"{person_name}_original.jpg")
        img = load_img(original_image_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)  # Ajustar la imagen para el generador

        # Crear 20 imágenes aumentadas por cada imagen original
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=person_folder, save_prefix="aug", save_format="jpg"):
            i += 1
            if i >= 20:  # Ajusta el número de imágenes aumentadas si es necesario
                break

print("Dataset aumentado creado.")

# OpenCV para codificación de los rostros

Usaremos para poder reconocer rostros en imágenes y videos, primero necesito codificar (o cuantificar) los rostros en mi conjunto de entrenamiento. Es importante destacar que en este caso no se va entrenar una red neuronal desde cero: la red (en la libreria face_recognition) ya ha sido entrenada para generar embeddings de 128 dimensiones a partir de un conjunto de datos de aproximadamente 3 millones de imágenes.

Obviamente se podría entrenar una red desde cero o incluso ajustar los pesos de un modelo existente, pero eso demandaria mucho tiempo.
Además, necesitaría una gran cantidad de imágenes para entrenar la red desde cero. En su lugar, es más sencillo utilizar la red preentrenada y emplearla para construir embeddings de 128 dimensiones para cada una mis imagenes de mi conjunto de datos.


Para clasificar cada rostro, estoy usando un modelo llamado KNN (o K-Nearest Neighbors), que es una forma de aprendizaje automático que compara caras para encontrar las más parecidas.

Así es como funciona: cuando el modelo ve un nuevo rostro, busca en su "memoria" (es decir, su conjunto de datos) los rostros más parecidos a ese nuevo rostro. Luego, hace un "sistema de votos" con esos rostros cercanos para decidir quién es. Imagina que los rostros cercanos “votan” por quién creen que es el nuevo rostro, y la opción con más votos es el resultado final.

Este método de KNN es solo una forma de hacerlo. Existen otros modelos de machine learning que también podrían usarse para hacer la misma tarea de clasificación de rostros.

In [1]:
#importación de paquetes necesarios
from imutils import paths
import face_recognition
import pickle
import cv2
import os
import time

args = {}
args['dataset'] = os.getcwd() + '\\dataset'               # Ruta al directorio de entrada de rostros e imágenes
args['encodings'] = os.getcwd() + '\\encodings.pickle'    # Ruta a la base de datos serializada de codificaciones faciales
args['detection_method'] = 'cnn'                          # Modelo de detección facial a utilizar: el método CNN es más preciso pero lento.
os.getcwd()

'd:\\proyecto_final\\face'

# Crear embeddings faciales

Un embedding facial es como una "huella digital" especial para cada cara, hecha de números en lugar de líneas. Cuando la computadora ve una cara, convierte esa cara en un grupo de números únicos que representan cómo se ve esa persona.

Esos números son como una “tarjeta de identificación” para cada rostro: si ve una cara similar, puede comparar sus "huellas" de números para ver si son parecidas. 

In [2]:
def create_facial_embeddings(args):
    # obtener las rutas de las imágenes de entrada en nuestro conjunto de datos
    print('[INFO] cuantificando rostros...')
    imagePaths = list(paths.list_images(args['dataset']))
    # inicializar la lista de codificaciones conocidas y nombres conocidos
    knownEncodings = []
    knownNames = []
    for (i, imagePath) in enumerate(imagePaths):
        print(i, imagePath)

    # OpenCV ordena los canales de color en BGR, pero dlib en realidad espera RGB. El módulo face_recognition usa dlib, por lo que necesitamos cambiar el espacio de color y nombrar la nueva imagen rgb
    ti = time.time()
    print('[INFO] procesando imagen...')
    # bucle sobre las rutas de las imágenes
    for (i, imagePath) in enumerate(imagePaths):
        # extraer el nombre de la persona de la ruta de la imagen
        print('{}/{}'.format(i+1, len(imagePaths)), end=', ')
        name = imagePath.split(os.path.sep)[-2]
        # cargar la imagen de entrada y convertirla de BGR (orden de OpenCV) al orden de dlib (RGB)
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # detectar las coordenadas (x,y) de las cajas delimitadoras correspondientes a cada rostro en la imagen de entrada
        boxes = face_recognition.face_locations(rgb, model=args['detection_method'])
        # calcular el embedding facial para el rostro, es decir, convertir las cajas delimitadoras del rostro en una lista de 128 números
        encodings = face_recognition.face_encodings(rgb, boxes)
        # bucle sobre las codificaciones
        for encoding in encodings:
            # agregar cada codificación + nombre a nuestro conjunto de nombres y codificaciones conocidos
            knownEncodings.append(encoding)
            knownNames.append(name)
    print('¡Listo!')
    print('Tiempo tomado: {:.1f} minutos'.format((time.time() - ti)/60))

    # volcar los nombres y codificaciones al disco para un futuro uso
    # encodings.pickle contiene los embeddings de rostros en 128 dimensiones para cada rostro en nuestro conjunto de datos
    print('[INFO] serializando codificaciones...')
    data = {'encodings': knownEncodings, 'names': knownNames}
    f = open(args['encodings'], 'wb')
    f.write(pickle.dumps(data))
    f.close()
    print('¡Listo!')

In [3]:
# Usando solo el CPU !!
args = {}
args['dataset'] = os.getcwd() + '\\dataset'               # Ruta al directorio de entrada de rostros e imágenes
args['encodings'] = os.getcwd() + '\\encodings.pickle'    # Ruta a la base de datos serializada de codificaciones faciales
args['detection_method'] = 'cnn'                          # Modelo de detección facial a utilizar: el método CNN es más preciso pero lento.

create_facial_embeddings(args)

[INFO] quantifying faces...
0 d:\proyecto_final\face\dataset\brandon\brandon(1).jpg
1 d:\proyecto_final\face\dataset\brandon\brandon(10).jpg
2 d:\proyecto_final\face\dataset\brandon\brandon(2).jpg
3 d:\proyecto_final\face\dataset\brandon\brandon(3).jpg
4 d:\proyecto_final\face\dataset\brandon\brandon(4).jpg
5 d:\proyecto_final\face\dataset\brandon\brandon(5).jpg
6 d:\proyecto_final\face\dataset\brandon\brandon(6).jpg
7 d:\proyecto_final\face\dataset\brandon\brandon(7).jpg
8 d:\proyecto_final\face\dataset\brandon\brandon(8).jpg
9 d:\proyecto_final\face\dataset\brandon\brandon(9).jpg
10 d:\proyecto_final\face\dataset\gabriel\gabriel(1).jpg
11 d:\proyecto_final\face\dataset\gabriel\gabriel(10).jpg
12 d:\proyecto_final\face\dataset\gabriel\gabriel(2).jpg
13 d:\proyecto_final\face\dataset\gabriel\gabriel(3).jpg
14 d:\proyecto_final\face\dataset\gabriel\gabriel(4).jpg
15 d:\proyecto_final\face\dataset\gabriel\gabriel(5).jpg
16 d:\proyecto_final\face\dataset\gabriel\gabriel(6).jpg
17 d:\proye

# Reconocer rostros en imágenes

In [4]:
import face_recognition
import pickle
import cv2
import os
import time
from collections import Counter
import numpy as np

args = {}
args['encodings'] = os.getcwd() + '\\encodings.pickle'        # Ruta a la base de datos serializada de codificaciones faciales
args['image'] = os.getcwd() + '\\image_test\\test (1).jpg'    # Ruta a la imagen de entrada
args['detection_method'] = 'cnn'                              # Modelo de detección facial a utilizar: el método CNN es más preciso pero lento. 

In [5]:
def recognise_faces(args):
    ti = time.time()
    # cargar los rostros y embeddings conocidos
    print('[INFO] cargando codificaciones...')
    data = pickle.loads(open(args['encodings'], 'rb').read())
    # cargar la imagen de entrada y convertirla de BGR a RGB
    image = cv2.imread(args['image'])
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # detectar las coordenadas (x,y) de las cajas delimitadoras correspondientes a cada rostro en la imagen de entrada, luego calcular los embeddings faciales para cada rostro
    print('[INFO] reconociendo rostros...')
    boxes = face_recognition.face_locations(rgb, model=args['detection_method'])
    encodings = face_recognition.face_encodings(rgb, boxes)
    # inicializar la lista de nombres para cada rostro detectado
    names = []
    confidences = []

    # bucle sobre los embeddings faciales
    for encoding in encodings:
        # intentar hacer coincidir cada rostro en la imagen de entrada con nuestras codificaciones conocidas, la función devuelve una lista de valores Verdadero/Falso, uno para cada codificación conocida
        # calcular las distancias entre el encoding detectado y todas las codificaciones conocidas
        distances = face_recognition.face_distance(data['encodings'], encoding)
        # encontrar la coincidencia más cercana
        best_match_index = np.argmin(distances)
        best_distance = distances[best_match_index]
        
        # definir un umbral para considerar una coincidencia
        threshold = 0.6  # Ajusta este valor según tus necesidades

        # calcular la "precisión" como un porcentaje de confianza basado en la distancia
        if best_distance < threshold:
            accuracy = (1 - best_distance / threshold)
            name = data['names'][best_match_index]
        else:
            accuracy = 0
            name = 'Desconocido'

        # actualizar la lista de nombres
        names.append(name)
        confidences.append(accuracy)

    print([' '.join([e.title() for e in names])])
    print('Tiempo tomado: {:.1f} segundos'.format(time.time() - ti))
          
   # Visualizar con cajas delimitadoras, nombres y porcentajes de confianza
    for ((top, right, bottom, left), name, confidence) in zip(boxes, names, confidences):
        # dibujar la caja delimitadora
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        display_text = f"{name}: {confidence:.2f}%" if confidence > 0 else name
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image, display_text, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # mostrar el cuadro resultante, presiona 'q' para salir
    window_text = args['image'].split(os.path.sep)[-1]
    cv2.imshow(window_text, image)
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    # guardar la imagen de salida
    cv2.imwrite(args['image'].rsplit('.', 1)[0] + '_output.jpg', image)


In [6]:
args['image'] = os.getcwd() + '\\image_test\\test (1).jpg'
recognise_faces(args)

[INFO] loading encodings...
[INFO] recognising faces...
['Jeyson Noe Juan Jovani Brandon Brandon']
Time taken: 491.6 seconds


In [61]:
args['image'] = os.getcwd() + '\\image_test\\test (2).jpg'
recognise_faces(args)

[INFO] loading encodings...
[INFO] recognising faces...
['Saul', 'Gabriel', 'Jeyson', 'Juan', 'Saul']
Time taken: 121.3 seconds


In [8]:
args['image'] = os.getcwd() + '\\image_test\\test (3).jpg'
recognise_faces(args)

[INFO] loading encodings...
[INFO] recognizing faces...
['Unknown', 'Claire Dearing', 'Owen Grady', 'Unknown']
Time taken: 25.0 seconds


In [9]:
args['image'] = os.getcwd() + '\\image_test\\test (4).jpg'
recognise_faces(args)

[INFO] loading encodings...
[INFO] recognizing faces...
['Alan Grant']
Time taken: 20.3 seconds


In [10]:
args['image'] = os.getcwd() + '\\image_test\\test (5).jpg'
recognise_faces(args)

[INFO] loading encodings...
[INFO] recognizing faces...
['Ian Malcolm']
Time taken: 154.1 seconds


In [11]:
args['image'] = os.getcwd() + '\\image_test\\test (6).jpg'
recognise_faces(args)

[INFO] loading encodings...
[INFO] recognizing faces...
['Alan Grant', 'Ian Malcolm', 'Ellie Sattler']
Time taken: 41.8 seconds


# Reconocer rostros en un archivo de video

In [1]:
# Paquetes necesarios 
import face_recognition  # Biblioteca principal para la detección y reconocimiento de rostros.
import imutils  # Facilita operaciones comunes de procesamiento de imágenes, como redimensionar y rotar fotogramas de video, optimizando el procesamiento y haciendo que los fotogramas tengan un tamaño adecuado.
import pickle  # Biblioteca utilizada para serializar y deserializar datos (guardar y cargar). 
import cv2  # Biblioteca OpenCV, fundamental para leer, procesar y mostrar cada fotograma del video, además de realizar conversiones de color y dibujar cuadros delimitadores alrededor de los rostros.
import os  # Permite interactuar con el sistema de archivos, como manejar rutas y directorios.
import time  # Utilizada para medir el tiempo de procesamiento.
from collections import Counter  # Herramienta para contar la frecuencia de aparición de rostros reconocidos, ayudando a identificar qué rostros aparecen con más frecuencia en el video.


args = {}
args['encodings'] = os.getcwd() + '\\encodings.pickle'              # ruta a la base de datos serializada de codificaciones faciales
args['input'] = os.getcwd() + '\\video_test\\trailer.mp4'           # ruta al video de entrada
args['output'] = args['input'].rsplit('.', 1)[0] + '_outputt.avi'   # ruta al video de salida
args['display'] = 1                                                 # mostrar el cuadro de salida en pantalla: sí o no
args['detection_method'] = 'hog'                                    # modelo de detección facial a utilizar: el método CNN es más preciso pero más lento. HOG es más rápido pero menos preciso.

# Ahora usaremos hog ya que solo estoy usando mi cpu.

In [3]:
def recognise_faces_video(args):
    ti = time.time()
    # cargar los rostros y embeddings conocidos
    print('[INFO] cargando codificaciones...')
    data = pickle.loads(open(args['encodings'], 'rb').read())
    # inicializar el puntero al archivo de video y el escritor de video
    print('[INFO] procesando video...')
    stream = cv2.VideoCapture(args['input'])
    writer = None    # opcionalmente escribiendo cuadros de video procesados en el disco más tarde, por lo que inicializamos writer en None

    # bucle sobre los cuadros del flujo de archivos de video
    while True:
        # obtener el siguiente cuadro
        (grabbed, frame) = stream.read()
        # si no se obtuvo el cuadro, hemos llegado al final del flujo
        if not grabbed:
            break
        # convertir el cuadro de entrada de BGR a RGB y luego redimensionarlo a un ancho de 750px (para acelerar el procesamiento)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb = imutils.resize(frame, width=750)
        r = frame.shape[1] / float(rgb.shape[1])
        # detectar las coordenadas (x,y) de las cajas delimitadoras correspondientes a cada rostro en el cuadro de entrada, luego calcular los embeddings faciales para cada rostro
        boxes = face_recognition.face_locations(rgb, model=args['detection_method'])
        encodings = face_recognition.face_encodings(rgb, boxes)
        names = []

        # bucle sobre los embeddings faciales
        for encoding in encodings:
            # intentar hacer coincidir cada rostro en la imagen de entrada con nuestras codificaciones conocidas, la función devuelve una lista de valores Verdadero/Falso, uno para cada codificación conocida
            # Internamente, la función compare_faces está calculando la distancia euclidiana entre el embedding candidato y todos los rostros en nuestras codificaciones conocidas
            votes = face_recognition.compare_faces(data['encodings'], encoding)
            # verificar si se encontró una coincidencia
            if True in votes:
                # encontrar los nombres correspondientes de todos los rostros coincidentes (vote==True)
                matches = [name for name, vote in list(zip(data['names'], votes)) if vote == True]  
                # determinar el nombre que ocurre con mayor frecuencia (nota: en el improbable caso de un empate, Python seleccionará la primera entrada en el diccionario)
                name = Counter(matches).most_common()[0][0]
            else:
                name = 'Desconocido'
            # actualizar la lista de nombres
            names.append(name)

        # visualizar con cajas delimitadoras y nombres etiquetados, recorrer los rostros reconocidos
        for ((top, right, bottom, left), name) in zip(boxes, names):
            # reescalar las coordenadas del rostro
            top = int(top * r)
            right = int(right * r)
            bottom = int(bottom * r)
            left = int(left * r)
            # dibujar el nombre del rostro predicho en la imagen
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

        # si el escritor de video es None *Y* se proporciona la ruta de salida (para escribir el cuadro en el disco)
        if writer is None and args['output'] is not None:
            fourcc = cv2.VideoWriter_fourcc(*'MJPG')    # para usar el código de 4 caracteres “MJPG”
            writer = cv2.VideoWriter(args['output'], fourcc, 24, (frame.shape[1], frame.shape[0]), True)    # ruta del archivo de salida, fourcc, objetivo de fotogramas por segundo y dimensiones del cuadro
        # si el escritor no es None, escribir el cuadro con rostros reconocidos en el disco
        if writer is not None:
            writer.write(frame)

        # verificar si se muestra el cuadro de salida en pantalla
        if args['display'] == 1:
            cv2.imshow('Video file', frame)
            # si se presiona la tecla `q`, salir del bucle
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # hacer algo de limpieza
    cv2.destroyAllWindows()
    stream.release()    # cerrar los punteros del archivo de video
    # verificar si es necesario liberar el puntero del escritor de video
    if writer is not None:
        writer.release()
    print('Tiempo tomado: {:.1f} minutos'.format((time.time() - ti)/60))

In [4]:
args['input'] = os.getcwd() + '\\video_test\\trailer.mp4'
args['output'] = args['input'].rsplit('.', 1)[0] + '_outputt.avi'
recognise_faces_video(args)

[INFO] loading encodings...
[INFO] processing video...
Time taken: 15.2 minutes


# Reconocer rostros en la cámara web

In [17]:
import imutils
from imutils import paths
from imutils.video import VideoStream
import face_recognition
import pickle
import cv2
import os
import time
from collections import Counter

In [29]:
#create facial embeddings
args = {}
args['dataset'] = os.getcwd() + '\\dataset_webcam'               
args['encodings'] = os.getcwd() + '\\encodings_webcam.pickle'   
args['detection_method'] = 'cnn'                                 

create_facial_embeddings(args)

[INFO] quantifying faces...
0 d:\proyecto_final\face\dataset_webcam\james\Screenshot_8.jpg
1 d:\proyecto_final\face\dataset_webcam\Jhonatan\Jhonatan.jpg
[INFO] processing image...
1/2, 2/2, Done!
Time taken: 2.9 minutes
[INFO] serializing encodings...
Done!


In [30]:
# Encendido de la camara
args = {}
args['encodings'] = os.getcwd() + '\\encodings_webcam.pickle'    
args['output'] = os.getcwd() + '\\webcam_test\\output.avi'       
args['display'] = 1                                              
args['detection_method'] = 'hog' 

#hog por mi cpu

In [31]:
ti = time.time()
# cargar los rostros y embeddings conocidos
print('[INFO] cargando codificaciones...')
data = pickle.loads(open(args['encodings'], 'rb').read())
# inicializar el flujo de video y el puntero al archivo de video de salida, luego permitir que el sensor de la cámara se caliente
print('[INFO] iniciando flujo de video...')
vs = VideoStream(src=0).start()    # usar VideoStream para acceder a la cámara web, usar src=1 para una segunda cámara web
time.sleep(2.0)    # time.sleep con 2 segundos para calentar la cámara web
writer = None    # opcionalmente escribiendo cuadros de video procesados en el disco más tarde, por lo que inicializamos writer en None

# bucle sobre los cuadros del flujo de video
while True:
    # obtener un cuadro del flujo de video en paralelo
    frame = vs.read()
    # convertir el cuadro de entrada de BGR a RGB y luego redimensionarlo a un ancho de 750px (para acelerar el procesamiento)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb = imutils.resize(frame, width=750)
    r = frame.shape[1] / float(rgb.shape[1])
    # detectar las coordenadas (x,y) de las cajas delimitadoras correspondientes a cada rostro en el cuadro de entrada, luego calcular los embeddings faciales para cada rostro
    boxes = face_recognition.face_locations(rgb, model=args['detection_method'])
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []
    # bucle sobre los embeddings faciales
    for encoding in encodings:
        # intentar hacer coincidir cada rostro en la imagen de entrada con nuestras codificaciones conocidas, la función devuelve una lista de valores Verdadero/Falso, uno para cada codificación conocida
        # Internamente, la función compare_faces está calculando la distancia euclidiana entre el embedding candidato y todos los rostros en nuestras codificaciones conocidas
        votes = face_recognition.compare_faces(data['encodings'], encoding)
        # verificar si se encontró una coincidencia
        if True in votes:
            # encontrar los nombres correspondientes de todos los rostros coincidentes (vote==True)
            matches = [name for name, vote in list(zip(data['names'], votes)) if vote == True]  
            # determinar el nombre que ocurre con mayor frecuencia (nota: en el improbable caso de un empate, Python seleccionará la primera entrada en el diccionario)
            name = Counter(matches).most_common()[0][0]
        else:
            name = 'Desconocido'
        # actualizar la lista de nombres
        names.append(name)

    # visualizar con cajas delimitadoras y nombres etiquetados, recorrer los rostros reconocidos
    for ((top, right, bottom, left), name) in zip(boxes, names):
        # reescalar las coordenadas del rostro
        top = int(top * r)
        right = int(right * r)
        bottom = int(bottom * r)
        left = int(left * r)
        # dibujar el nombre del rostro predicho en la imagen
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # si el escritor de video es None *Y* se proporciona la ruta de salida (para escribir el cuadro en el disco)
    if writer is None and args['output'] is not None:
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')    # para usar el código de 4 caracteres “MJPG”
        writer = cv2.VideoWriter(args['output'], fourcc, 20, (frame.shape[1], frame.shape[0]), True)    # ruta del archivo de salida, fourcc, objetivo de fotogramas por segundo y dimensiones del cuadro
    # si el escritor no es None, escribir el cuadro con rostros reconocidos en el disco
    if writer is not None:
        writer.write(frame)
        
    # verificar si se muestra el cuadro de salida en pantalla
    if args['display'] == 1:
        cv2.imshow('Webcam', frame)
        # si se presiona la tecla `q`, salir del bucle
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
# hacer algo de limpieza
cv2.destroyAllWindows()
vs.stop()
# verificar si es necesario liberar el puntero del escritor de video
if writer is not None:
    writer.release()
print('¡Listo! \nTiempo tomado: {:.1f} minutos'.format((time.time() - ti)/60))

[INFO] loading encodings...
[INFO] starting video stream...
Done! 
Time taken: 0.7 minutes
